# Import dependencies

In [ ]:
import pandas as pd
import numpy as np

# Getting data
Get dataset

In [ ]:
# Get data as dataframe
data = pd.read_csv("../data/adult.data.csv")

data.head()

Drop rows with ? symbol

In [ ]:
# Create a mask indicating rows containing '?' symbols
mask = np.any(data == "?", axis=1)

# Select rows that do not contain '?' symbols
data = data[~mask]

Get info

In [ ]:
data.info()

Gat summary

In [ ]:
data.describe()

# Tasks

1. How many man and woman are in dataset (by sex feature)?

In [ ]:
data.groupby(["sex"]).size()

2. Woman mean age (by age feature)?

In [ ]:
female_age_avg = data.groupby(["sex"]).get_group("Male")["age"].mean().astype("int32")

print(f"Average female age is: {female_age_avg} year(s)")

3. German citizen ratio (by native-country feature)?

In [ ]:
german_person_count = data.groupby(["native-country"]).get_group("Germany").shape[0]

german_person_ratio = german_person_count / data.shape[0]

print(f"Ratio of persons from Germany is: {german_person_ratio:.3f}.")

4. Age mean and square root mean of persons who earns at least 50K per year ('salary') and for those who earns less?

In [ ]:
# Get groups by salary
reach_person_group = data.groupby(["salary"]).get_group(">50K")
other_person_group = data.groupby(["salary"]).get_group("<=50K")

# Get mean ages
reach_person_age_mean = reach_person_group["age"].mean()
other_person_age_mean = other_person_group["age"].mean()

# Get std
reach_person_age_std = reach_person_group["age"].std()
other_person_age_std = other_person_group["age"].std()

age_format = ".0f"
print(
    f"Reach persons have mean age {reach_person_age_mean:.0f}\
 with std {reach_person_age_std:.2f}"
)
print(
    f"Other persons have mean age {other_person_age_mean:.0f}\
 with std {other_person_age_std:.2f}"
)

6. Is it true that persons who earns more than 50K have higher education?
(Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate)

In [ ]:
# Set check list of education level
higher_education_list = [
    "Bachelors",
    "Prof-school",
    "Assoc-acdm",
    "Assoc-voc",
    "Masters",
    "Doctorate",
]

# Check condition
reach_means_smart = reach_person_group["education"].isin(higher_education_list).all()

print(f"Answer on this question is: {reach_means_smart}.")

7. Age statistics for each race and sex using 'groupby' and 'describe'.

In [ ]:
# Get group stat
race_age_stat = data.groupby(["race", "sex"])["age"].describe()

race_age_stat

Max age of Amer-Indian-Eskimo man.

In [ ]:
# Get Amer-Indian-Eskimo male max age
max_age = race_age_stat["max"]["Amer-Indian-Eskimo"]["Male"]

print(f"\nMax age of Amer-Indian-Eskimo is: {max_age:.0f}")

8. Which category of persons earns at least 50K: married or not (marital-status = (Married-civ-spouse, Married-spouse-absent или Married-AF-spouse)

In [ ]:
# Set list of statuses which are equal to married
married_statuses = ["Married-civ-spouse", "Married-spouse-absent", "Married-AF-spouse"]

# Get mask for married persons
data["Married"] = data["marital-status"].isin(married_statuses)

# Get ratios
married_ratio = (
    data[data["Married"]]["salary"].value_counts(normalize=True).get(">50K", 0)
)
single_ratio = (
    data[~data["Married"]]["salary"].value_counts(normalize=True).get(">50K", 0)
)

if married_ratio > single_ratio:
    print("Reach person ratio is bigger for married persons.")
elif married_ratio < single_ratio:
    print("Reach person ratio is bigger for single persons.")
else:
    print("Ratios are identical.")

9. Maximal working hours per week (hours-per-week)? Haw many persons work these hours and how many reach persons among them?

In [ ]:
max_hours_per_week = data["hours-per-week"].max()
person_count = (data["hours-per-week"] == max_hours_per_week).sum()

data["hard_worker"] = data["hours-per-week"] == max_hours_per_week

reach_percentage = (
    data[data["hard_worker"]]["salary"].value_counts(normalize=True).get(">50K", 0)
    * 100
)

print(
    f"Max hours per week is {max_hours_per_week}. {person_count} persons do it and {reach_percentage:.1f}% of them are reach persons."
)

10. Average working hours (hours-per-week) for poor and reach persons (salary) for each country (native-country).

In [ ]:
average_hours_per_week = data.groupby(["native-country", "salary"])[
    "hours-per-week"
].mean()
average_hours_per_week